In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPRegressor
from datetime import datetime as dt
from IPython.display import clear_output, display
#import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import wilcoxon, ttest_ind
from IPython.display import display
import matplotlib
import sys
from scipy.stats import kruskal, f_oneway
from statistics import median
from scipy.stats import wilcoxon, ttest_ind, mannwhitneyu

np.set_printoptions(threshold=sys.maxsize)

BASE_DIR = "/content/drive/MyDrive/Teseo/CCCP"

# Data extraction and preliminar preprocessing

In [ ]:
# DEMOGRAPHIC
df = pd.read_excel(BASE_DIR + 'CCP_dati_cadute_stroke_20211103.xlsx',
                   sheet_name='Anagrafica',
                   skiprows=[0],
                   dtype={'Id paziente (H-Opera) ': str}

                  )

ID = df['Id paziente (H-Opera) ']
eta = df['Eta\'']
cogn = df['Decadimeno Cognitivo?'].eq('SI').mul(1)
genere = df["Sesso"].eq('Maschio').mul(1)
anagrafica = pd.concat([ID , eta, cogn, genere], axis=1)
print(anagrafica.shape)
anagrafica.head(5)

(166, 4)


,Id paziente (H-Opera),Eta',Decadimeno Cognitivo?,Sesso
0,2011005654,63.0,0,1
1,2014005464,47.0,0,1
2,2014008730,86.0,0,1
3,2014009118,63.0,0,1
4,2016004931,60.0,0,1


In [ ]:
# FALLS
df = pd.read_excel(BASE_DIR + 'CCP_questionari_cadute_n.xlsx',
                   sheet_name='SINTESI_NUMERO_cadute',
                   skiprows=[0],
                   dtype={'Id paziente (H-Opera) ': str})
cadute = df[["Id paziente (H-Opera) ", "caduto almeno una volta"]].iloc[:166]
print(cadute.shape)

print("\nSoggetti che hanno sperimentato almeno una caduta: {0}".format(str((cadute['caduto almeno una volta'] >= 1).sum())))
print(cadute.head(5))

(166, 2)

Soggetti che hanno sperimentato almeno una caduta: 55
  Id paziente (H-Opera)   caduto almeno una volta
0             PZ00728521                      0.0
1             PZ00019652                      0.0
2             PZ00118046                      1.0
3             PZ01386247                      0.0
4             2017005632                      1.0


In [ ]:
# EXCEL DATA (Clinical tests)

sheet_data_t0 = pd.read_excel(BASE_DIR + 'CCP_dati_cadute_stroke_20211103.xlsx',
                   sheet_name='Valutazioni_T0',
                   skiprows=[0],
                   dtype={'Id paziente (H-Opera) ': str}

                  )

sheet_data_t0 = sheet_data_t0.drop(['Data Valutazione (T0)'], axis=1)

sheet_data_t0['Ausili durante il TUG (nessuno/bastone/deambulatore)_T0'] = sheet_data_t0['Ausili durante il TUG (nessuno/bastone/deambulatore)_T0'].astype('category').cat.codes
sheet_data_t0['Ausili durante Test 10 m (nessuno/bastone/deambulatore)_T0'] = sheet_data_t0['Ausili durante Test 10 m (nessuno/bastone/deambulatore)_T0'].astype('category').cat.codes
sheet_data_t0['Ha il pannolone?_T0'] = sheet_data_t0['Ha il pannolone?_T0'].astype('category').cat.codes
sheet_data_t0['Controllo Vescica_T0'] = sheet_data_t0['Controllo Vescica_T0'].astype('category').cat.codes
sheet_data_t0['Controllo Alvo_T0'] = sheet_data_t0['Controllo Alvo_T0'].astype('category').cat.codes

print(sheet_data_t0.shape)

################

sheet_data_t1 = pd.read_excel(BASE_DIR + 'CCP_dati_cadute_stroke_20211103.xlsx',
                   sheet_name='Valutazione_T1',
                   skiprows=[0],
                   dtype={'Id paziente (H-Opera) ': str}

                  )

sheet_data_t1 = sheet_data_t1.drop(['Data Valutazione (T1)'], axis=1)

sheet_data_t1['Ausili durante il TUG (choice=Nessuno/deambulatore/bastone)_T1'] = sheet_data_t1['Ausili durante il TUG (choice=Nessuno/deambulatore/bastone)_T1'].astype('category').cat.codes
sheet_data_t1['durata TUG  - Ripetizione 1_T1'].astype(float)
sheet_data_t1['Ausili durante il Test 10 m (nessuno/bastone/deambulatore)_T1'] = sheet_data_t1['Ausili durante il Test 10 m (nessuno/bastone/deambulatore)_T1'].astype('category').cat.codes
sheet_data_t1['Ha il pannolone?_T1'] = sheet_data_t1['Ha il pannolone?_T1'].astype('category').cat.codes
sheet_data_t1['Controllo Vescica_T1'] = sheet_data_t1['Controllo Vescica_T1'].astype('category').cat.codes
sheet_data_t1['Controllo Alvo_T1'] = sheet_data_t1['Controllo Alvo_T1'].astype('category').cat.codes

print(sheet_data_t1.shape)

(166, 28)
(166, 28)


In [ ]:
# TUG data extraction and preprocessing (averaging over multiple repetitions of the tests)

def get_TUG_data(t):
    # ANALISI DATI TUG

    root_folder = BASE_DIR + "Dati_cadute/"
    patient_groups = os.listdir(root_folder)

    first_iteration = True

    columns_name = ["Id paziente (H-Opera) "]
    rows = []

    for i,group in enumerate(patient_groups):

        patients = os.listdir(root_folder + group)

        for j,patient in enumerate(patients):

            try:

                proc_data = root_folder + group + "/" + patient + "/TUG/proc_data"
                t01 = os.listdir(proc_data)
                # Perform analysis at T1
                a = dt.strptime(t01[0], "%d_%m_%Y")
                b = dt.strptime(t01[1], "%d_%m_%Y")

                t1 = t01[1] if a<b else t01[0]
                t0 = t01[0] if a<b else t01[1]

                tc = ""
                if t == "T0":
                    tc = t0
                else:
                    tc = t1

                t_folder = proc_data + "/" + tc
                trials = os.listdir(t_folder)

                mean_values = np.array([])

                for trial in trials:

                    trial = t_folder + "/" + trial
                    files = os.listdir(trial)

                    for filename in files:
                        if filename.find("csv") != -1:

                            data = pd.read_csv(trial + "/" + filename, sep=",", header=None)

                            if first_iteration:
                                columns_name += [x + "_" + t for x in list(data.iloc[:,0])]
                                first_iteration = False

                            if mean_values.size == 0:
                                mean_values = np.array(list(data.iloc[:,1].values)).copy()

                            else:
                                mean_values += np.array(list(data.iloc[:,1].values))

                        else:
                            continue


                mean_values /= len(trials)
                row = [patient] + list(mean_values)
                rows.append(row)

                clear_output(wait=True)
                print("group {0}, patient {1}".format(i+1,j+1))

            except:
                 continue


    TUG = pd.DataFrame(columns=columns_name)
    #TUG = TUG.append(pd.DataFrame(rows, columns=TUG.columns))
    TUG = pd.concat([TUG, pd.DataFrame(rows, columns=TUG.columns)], ignore_index=True)
    return TUG


def get_PSA_data(t):
    # ANALISI DATI PSA

    root_folder = "Dati_cadute/"
    patient_groups = os.listdir(root_folder)

    first_iteration = True

    columns_name = ["Id paziente (H-Opera) "]
    rows = []

    for i, group in enumerate(patient_groups):

        patients = os.listdir(root_folder + group)

        for j,patient in enumerate(patients):

            try:

                proc_data = root_folder + group + "/" + patient + "/PSA/proc_data"
                t01 = os.listdir(proc_data)
                # Perform analysis at T1
                a = dt.strptime(t01[0], "%d_%m_%Y")
                b = dt.strptime(t01[1], "%d_%m_%Y")

                t1 = t01[1] if a<b else t01[0]
                t0 = t01[0] if a<b else t01[1]

                tc = ""
                if t == "T0":
                    tc = t0
                else:
                    tc = t1

                t_folder = proc_data + "/" + tc
                trials = os.listdir(t_folder)

                mean_values = np.array([])
                for trial in trials[:5]: #Quiet Stance Test solo con occhi aperti

                    trial = t_folder + "/" + trial
                    files = os.listdir(trial)

                    for filename in files:
                        if filename.find("csv") != -1:

                            data = pd.read_csv(trial + "/" + filename, sep=",", header=None)

                            if first_iteration:
                                columns_name += list(data.iloc[:,0])
                                first_iteration = False

                            if mean_values.size == 0:
                                mean_values = np.array(list(data.iloc[:,1].values)).copy()

                            else:
                                mean_values += np.array(list(data.iloc[:,1].values))

                        else:
                            continue

                #mean_values /= len(trials)
                mean_values /= 5.0
                row = [patient] + list(mean_values)
                rows.append(row)

                clear_output(wait=True)
                print("group {0}, patient {1}".format(i+1,j+1))

            except:
                 continue

    PSA = pd.DataFrame(columns=columns_name)
    PSA = PSA.append(pd.DataFrame(rows, columns=PSA.columns))
    return PSA

TUG_t1 = get_TUG_data("T1")

group 2, patient 86


<ipython-input-6-2ac70cf92d12>:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  TUG = pd.concat([TUG, pd.DataFrame(rows, columns=TUG.columns)], ignore_index=True)


# Patient Ranking

## Build Datasets

### Full (iTUG + clinical features)

In [ ]:
# Sample dataset from our data
merged1 = pd.merge(anagrafica, sheet_data_t1, on="Id paziente (H-Opera) ")
merged2 = pd.merge(merged1, TUG_t1, on="Id paziente (H-Opera) ")
dataset = pd.merge(merged2, cadute, on="Id paziente (H-Opera) ")

dataset = dataset.dropna().reset_index(drop=True)
features_list = list(dataset.columns[31:-1])
final_list = [s.split('_')[0] for s in features_list]
df = pd.DataFrame(final_list, columns=['iTUG Features'])

dataset.shape

(127, 132)

### iTUG only

In [ ]:
# Sample dataset from our data
tmp_merged = pd.merge(anagrafica, TUG_t1, on="Id paziente (H-Opera) ")
tmp_dataset = pd.merge(tmp_merged, cadute, on="Id paziente (H-Opera) ").dropna()

original_columns = set(list(dataset.columns))
itug_columns = set(list(tmp_dataset.columns))
diff_columns = list(original_columns.difference(itug_columns))

dataset_itug = dataset.copy()
dataset_itug = dataset_itug.drop(diff_columns, axis=1)
dataset_itug.shape

(127, 105)

## Feature Extraction Algorithm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def cross_validate(model,X,T):
    folds = 7
    scores = cross_val_score(model, X, T, cv=folds)
    return scores.mean()

def detect_relevant_features(in_dataset, iterations, threshold):

    relevant_counter = {}
    significant_model_counter = 0

    for i in range(iterations):

        clear_output(wait=True)
        print("Iteration {0}...".format(i))

        # Bilanciamento del dataset - Campioniamo ~ pazienti senza cadute
        df_non_caduti = in_dataset[in_dataset['caduto almeno una volta'] == 0].sample((in_dataset['caduto almeno una volta'] == 1).sum())
        df_caduti = in_dataset[in_dataset['caduto almeno una volta'] == 1]

        dataset = pd.concat([df_non_caduti, df_caduti], axis=0, ignore_index=True)
        dataset = dataset.sample(frac=1).reset_index(drop=True)

        # Compute relevant features for this sampling
        fallers = dataset[dataset['caduto almeno una volta'] == 1]
        non_fallers = dataset[dataset['caduto almeno una volta'] == 0]

        relevant = []
        features = list(dataset.columns)[1:-1]
        for feature in features:
            c1 = non_fallers[feature]
            c2 = fallers[feature]

            t_stat, t_p = ttest_ind(c1,c2)
            w_stat, w_p = wilcoxon(c1,c2)

            if(t_p < threshold or w_p < threshold):
                relevant.append(feature)

        if len(relevant) >= 1:

            # Restrict analysis to relevant features
            relevant.append("caduto almeno una volta")
            dataset2 = dataset[relevant]

            # Prepare dataset
            X = dataset2.iloc[:,:-1]
            y = dataset2['caduto almeno una volta']

            scaler = StandardScaler().fit(X)
            X_std = scaler.transform(X)

            kernels = ['linear', 'rbf']
            C_values = [0.1, 0.5, 1, 5, 10, 20, 50]

            # Train models
            model_scores = []
            for k in kernels:
                for c in C_values:
                    model = SVC(kernel=k, C=c)
                    model_scores.append(cross_validate(model, X_std, y))

            # If at least one model is "significant", add the current features to the relevant ones
            if(any(i >= 0.8 for i in model_scores)):
                significant_model_counter += 1
                for feature in relevant[:-1]:
                    if feature in relevant_counter:
                        relevant_counter[feature] += 1
                    else:
                        relevant_counter[feature] = 1

    return significant_model_counter, relevant_counter


def find_closest_index(ordered_list, new_value):
    left = 0
    right = len(ordered_list) - 1

    while left <= right:
        mid = (left + right) // 2
        if ordered_list[mid] == new_value:
            return mid
        elif ordered_list[mid] < new_value:
            right = mid - 1
        else:
            left = mid + 1
    return left

# Data Preparation

### Train-Validation-Test Sets

In [ ]:
ratio_v = 0.18
ratio_t = 0.25

fallers = dataset[dataset['caduto almeno una volta'] == 1]
fallers_test = fallers.sample(frac=ratio_t)
fallers_remaining =  dataset.iloc[fallers.index.difference(fallers_test.index)]
fallers_validation = fallers_remaining.sample(frac=ratio_v)
fallers_training = dataset.iloc[fallers_remaining.index.difference(fallers_validation.index)]

non_fallers = dataset[dataset['caduto almeno una volta'] == 0]
non_fallers_test = non_fallers.sample(frac=ratio_t)
non_fallers_remaining = dataset.iloc[non_fallers.index.difference(non_fallers_test.index)]
non_fallers_validation = non_fallers_remaining.sample(frac=ratio_v)
non_fallers_training = dataset.iloc[non_fallers_remaining.index.difference(non_fallers_validation.index)]

dataset_training = dataset.iloc[list(non_fallers_training.index) + list(fallers_training.index)]
dataset_validation = dataset.iloc[list(non_fallers_validation.index) + list(fallers_validation.index)]
dataset_test = dataset.iloc[list(non_fallers_test.index) + list(fallers_test.index)]

print(dataset_training.shape)
print(dataset_validation.shape)
print(dataset_test.shape)
print("---")
print("Fallers/Non-fallers in training set")
print(np.count_nonzero(dataset_training['caduto almeno una volta'].values == 1))
print(np.count_nonzero(dataset_training['caduto almeno una volta'].values == 0))
print("---")
print("Fallers/Non-fallers in validation set")
print(np.count_nonzero(dataset_validation['caduto almeno una volta'].values == 1))
print(np.count_nonzero(dataset_validation['caduto almeno una volta'].values == 0))
print("Fallers/Non-fallers in test set")
print(np.count_nonzero(dataset_test['caduto almeno una volta'].values == 1))
print(np.count_nonzero(dataset_test['caduto almeno una volta'].values == 0))

(78, 132)
(17, 132)
(32, 132)
---
Fallers/Non-fallers in training set
24
54
---
Fallers/Non-fallers in validation set
5
12
Fallers/Non-fallers in test set
10
22


### Data augmentation on training set

In [ ]:
import random
np.set_printoptions(threshold=sys.maxsize)

# Augment faller patients by applying noise to existing ones
training_fallers = dataset_training[dataset_training['caduto almeno una volta'] == 1].sample(n=15)

# Augmented training dataset
aug_dataset_training = dataset_training.copy()

# Routine to create fake data by applying noise to existing ones
new_rows = 15
for i in range(new_rows):
  row_i = training_fallers.iloc[i]
  new_row_dict = {}
  for col in training_fallers.columns:
    new_value = None
    # Special columns
    if col == "Id paziente (H-Opera) ":
      new_value = "PZ{0}".format(i)
    elif col == "Sesso":
      new_value = random.randint(0, 1)
    elif col == "caduto almeno una volta":
      new_value = 1
    elif col == "Ausili durante il TUG (choice=Nessuno/deambulatore/bastone)_T1":
      new_value = 2
    else:
      new_value = row_i[col] + np.random.normal(0.0, 1, size = 1)
    new_row_dict[col] = new_value
  aug_dataset_training= pd.concat([aug_dataset_training, pd.DataFrame(new_row_dict)], ignore_index=True)

print("Fallers/Non-fallers in augmented training set")
print(np.count_nonzero(aug_dataset_training['caduto almeno una volta'].values == 1))
print(np.count_nonzero(aug_dataset_training['caduto almeno una volta'].values == 0))

#aug_dataset_training

Fallers/Non-fallers in augmented training set
39
54


# Analysis with Clinical Data only

### Train-Validation Sets

In [ ]:
# Sample dataset from our data
merged1 = pd.merge(anagrafica, sheet_data_t1, on="Id paziente (H-Opera) ")
tmp_df = pd.merge(merged1, cadute, on="Id paziente (H-Opera) ")
tmp_df = tmp_df.dropna().reset_index(drop=True)
sheet_only_columns = list(tmp_df.columns)

dataset_t_only_training = aug_dataset_training[sheet_only_columns]
dataset_t_only_test = dataset_test[sheet_only_columns]

### Feature Extraction

In [ ]:
n = 1000
c_t_only, d_t_only =  detect_relevant_features(in_dataset=dataset_t_only_training, iterations=n, threshold=0.1)
relevant_features = dict(reversed(sorted(d_t_only.items(), key=lambda item: item[1])))

Iteration 999...


### Thresholds

In [ ]:
from scipy import stats

#### KEEP ONLY X% OF TOP MOST RELEVANT FEATURES

percent = 0.5
ordered_d_t_only = dict(reversed(sorted(d_t_only.items(), key=lambda item: item[1])))
keys_t_only = list(ordered_d_t_only.keys())

idx_t_only = find_closest_index(list(ordered_d_t_only.values()), int(c_t_only*percent))

new_columns_t_only = ['Id paziente (H-Opera) '] + keys_t_only[:idx_t_only] + ['caduto almeno una volta']
dataset_r_t_only = dataset_t_only_training[new_columns_t_only].copy()

# Determine order of each column by checking whether the higher ratio of fallers lies in the top or bottom half
ordering_t_only = []
for i,column in enumerate(new_columns_t_only):
    if i != 0 and i < len(new_columns_t_only) - 1:
        ordered = dataset_r_t_only.iloc[:,[i,-1]].sort_values(column, ascending=False).copy()
        fallers_in_head = np.count_nonzero(ordered.head(int(ordered.shape[0]/2)).iloc[:,1].values)
        fallers_in_tail = np.count_nonzero(ordered.tail(int(ordered.shape[0]/2)).iloc[:,1].values)
        if fallers_in_head < fallers_in_tail:
            ordering_t_only.append(False)
        else:
            ordering_t_only.append(True)
ordering_t_only = [False] + ordering_t_only

n_groups = 3
percentile = int(len(dataset_r_t_only) / n_groups)
percentiles = [percentile] * ((n_groups) - 1)
percentiles.append(len(dataset) - (n_groups-1)*percentile)

# Build dictionary of theshold values
thresholds_dict_list_t_only = {}
for i, var in enumerate(new_columns_t_only[:-1]):
    if i != 0:
        ordered = dataset_r_t_only.sort_values(var, ascending=ordering_t_only[i]).copy()
        d_i = {}

        # Th1 is the dividing value between 1 and 2 gorup
        th1 = dataset_r_t_only.sort_values(new_columns_t_only[i], ascending=ordering_t_only[i]).copy().iloc[percentiles[0]][new_columns_t_only[i]]
        # Th2 is the dividing value between 2 and 3 group
        th2 = dataset_r_t_only.sort_values(new_columns_t_only[i], ascending=ordering_t_only[i]).copy().iloc[2*percentiles[0]][new_columns_t_only[i]]

        d_i["Order"] = "Ascending" if ordering_t_only[i] else "Descending"
        d_i["Th1"] = th1
        d_i["Th2"] = th2
        thresholds_dict_list_t_only[var] = d_i


out_cols = ["Feature", "Rank1", "Rank2", "Rank3"]
out_df = pd.DataFrame(columns=out_cols)

rows = []
for key in thresholds_dict_list_t_only.keys():
    if thresholds_dict_list_t_only[key]["Order"] == "Descending":
        t1 = round(thresholds_dict_list_t_only[key]["Th1"],2)
        t2 = round(thresholds_dict_list_t_only[key]["Th2"],2)
        rows.append({"Feature": key, "Rank1": "x > {0}".format(t1), "Rank2": "{0} < x < {1}".format(t2, t1), "Rank3": "x < {0}".format(t2)})

for key in thresholds_dict_list_t_only.keys():
    if thresholds_dict_list_t_only[key]["Order"] == "Ascending":
        t1 = round(thresholds_dict_list_t_only[key]["Th1"],2)
        t2 = round(thresholds_dict_list_t_only[key]["Th2"],2)
        rows.append({"Feature": key, "Rank1": "x < {0}".format(t1), "Rank2": "{0} < x < {1}".format(t1, t2), "Rank3": "x > {0}".format(t2)})

out_df = pd.concat([out_df, pd.DataFrame(rows)], ignore_index=True)
#out_df.to_excel(BASE_DIR + "features_with_thresholds_clinical_data_only.xlsx", index=False)

### Test

In [ ]:
# For each test patient, perform classification and rank them
rankings = {}
for i in range(len(dataset_t_only_test)):
  # Get row
  row_i = dataset_t_only_test.iloc[i]
  patient_scores = []
  # For each column, determing whether score is R1, R2 or R3
  for col in dataset_t_only_test.columns:
    if col in thresholds_dict_list_t_only.keys():
      order = thresholds_dict_list_t_only[col]['Order']
      th1 = thresholds_dict_list_t_only[col]['Th1']
      th2 = thresholds_dict_list_t_only[col]['Th2']
      # Check membership to rank based on order and threshold values
      score = None
      value = row_i[col]
      if order == "Descending":
        if value > th1:
          score = 0
        elif value > th2:
          score = 1
        else:
          score = 2
      elif order == "Ascending":
        if value < th1:
          score = 0
        elif value < th2:
          score = 1
        else:
          score = 2
      patient_scores.append(score)
  rankings[row_i["Id paziente (H-Opera) "]] =   max(set(patient_scores), key=patient_scores.count)
v_rankings_list = []
for t in list(rankings.items()):
    v_rankings_list.append(list(t))
v_ranking_df = pd.DataFrame(v_rankings_list, columns=["Id paziente (H-Opera) ", "Ranking"])
dataset_validation_r_t_only = pd.merge(dataset_t_only_test, v_ranking_df, on='Id paziente (H-Opera) ')

In [ ]:
#print("Validation dataset results:\n")

r1 = dataset_validation_r_t_only[dataset_validation_r_t_only['Ranking'] == 0]
r1_f = r1[r1['caduto almeno una volta'] == 1]
r1_nf = r1[r1['caduto almeno una volta'] == 0]

#print("{0} patients classified as Rank#1: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r1), len(r1_f), np.round(100.0*np.round(len(r1_f)/len(r1),2)),2))
#print("---")

r2 = dataset_validation_r_t_only[dataset_validation_r_t_only['Ranking'] == 1]
r2_f = r2[r2['caduto almeno una volta'] == 1]
r2_nf = r2[r2['caduto almeno una volta'] == 0]

#print("{0} patients classified as Rank#2: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r2), len(r2_f), np.round(100.0*np.round(len(r2_f)/len(r2),2)),2))
#print("---")

r3 = dataset_validation_r_t_only[dataset_validation_r_t_only['Ranking'] == 2]
r3_f = r3[r3['caduto almeno una volta'] == 1]
r3_nf = r3[r3['caduto almeno una volta'] == 0]

#print("{0} patients classified as Rank#3: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r3), len(r3_f), np.round(100.0*np.round(len(r3_f)/len(r3),2)),2))

group_l = [
    np.round(100.0*np.round(len(r1_f)/len(r1),2)),
    np.round(100.0*np.round(len(r2_f)/len(r2),2)),
    np.round(100.0*np.round(len(r3_f)/len(r3),2))]
n_groups = 3

plt.figure(figsize=(9,6))
plt.title("Gold Standard Only - Risk of fall (Test set)", fontsize=20)
plt.bar([i for i in range(n_groups)], group_l)
plt.xticks([i for i in range(n_groups)], ["Rank {0}".format(i+1) for i in range(n_groups)], fontsize=20)
plt.yticks(fontsize=16)
plt.ylabel("Risk of Falling [%]", fontsize=20)
plt.ylim(0, 100)

for i in range(n_groups):
    plt.text(i - 0.2, group_l[i] + 5, str(group_l[i]) + "%",
            color = 'blue', fontweight = 'bold', fontsize=18)

plt.savefig(BASE_DIR + "risk_of_fall_gold_standard_only - validation.png", dpi=300)

# Analysis with iTUG only

### Train-Validation Sets

In [ ]:
dataset_training_itug = aug_dataset_training.drop(diff_columns, axis=1)
dataset_validation_itug = dataset_test.drop(diff_columns, axis=1)

print(dataset_training_itug.shape)
print(dataset_validation_itug.shape)

(93, 105)
(32, 105)


### Feature Extraction

In [ ]:
c_itug, d_itug =  detect_relevant_features(
    in_dataset=dataset_training_itug,
    iterations=0,
    threshold=0.1
)
itug_features = dict(reversed(sorted(d_itug.items(), key=lambda item: item[1])))

Iteration 999...


### Plots and Rankings

In [ ]:
from scipy import stats
from statistics import StatisticsError

#### KEEP ONLY X% OF TOP MOST RELEVANT FEATURES
percent = 0.5

ordered_d_itug = dict(reversed(sorted(d_itug.items(), key=lambda item: item[1])))
keys_itug = list(ordered_d_itug.keys())
idx_itug = int(len(ordered_d_itug.keys()) * percent)

print("Remaining relevant features: ")
print(idx_itug)

print("---------------\n")
print("Final list of relevant features: \n")
for item in keys_itug[:idx_itug]:
  print(item)
print("\n-----------------")
new_columns_itug = ['Id paziente (H-Opera) '] + keys_itug[:idx_itug] + ['caduto almeno una volta']

dataset_i_tug_r = dataset_training_itug[new_columns_itug].copy()

ordering_itug = []
for i,column in enumerate(new_columns_itug):
    if i != 0 and i < len(new_columns_itug) - 1:
        ordered = dataset_i_tug_r.iloc[:,[i,-1]].sort_values(column, ascending=False).copy()
        fallers_in_head = np.count_nonzero(ordered.head(39).iloc[:,1].values)
        fallers_in_tail = np.count_nonzero(ordered.tail(39).iloc[:,1].values)
        if fallers_in_head < fallers_in_tail:
            ordering_itug.append(False)
        else:
            ordering_itug.append(True)
ordering_itug = [False] + ordering_itug

k = {}
for patient in dataset_i_tug_r.iloc[:,0].values:
    k[patient] = []

n_groups = 3
percentile = int(len(dataset_i_tug_r) / n_groups)
percentiles = [percentile] * ((n_groups) - 1)
percentiles.append(len(dataset_i_tug_r) - (n_groups-1)*percentile)
#print(percentiles)

for i,var in enumerate(new_columns_itug[:-1]):
    if i != 0:
        ordered = dataset_i_tug_r.sort_values(var, ascending=ordering_itug[i]).copy()
        for j,patient in enumerate(ordered.iloc[:,0]):
            group = int(j/percentile)
            if group >= n_groups:
                group = n_groups - 1
            k[patient].append(group)

#print(d)
e = k.copy()
for w in k:
  k[w] = stats.mode(k[w])[0]
ll = []
for t in list(k.items()):
    ll.append(list(t))
cols = [new_columns_itug[0], 'Ranking']

ranking_df_itug = pd.DataFrame(ll, columns=cols)

merged_ranking_df_itug = pd.merge(dataset_i_tug_r, ranking_df_itug, on='Id paziente (H-Opera) ')
ordered_ranking_df_itug = merged_ranking_df_itug.sort_values('Ranking')

groups_l = []
for g in range(n_groups):
    c = np.count_nonzero(ordered_ranking_df_itug['Ranking'].values == g)
    groups_l.append(c)

x = []
partial= 0
for i, group in enumerate(range(n_groups)):
    b = 0
    if i == n_groups - 1:
        b = np.count_nonzero(ordered_ranking_df_itug['caduto almeno una volta'].values[partial:] == 1)
    else:
        b = np.count_nonzero(ordered_ranking_df_itug['caduto almeno una volta'].values[partial:partial + groups_l[i]] == 1)
    x.append(b)
    partial += groups_l[i]

t = groups_l
xv = np.array(x)

plt.figure(figsize=(9,6))
plt.title("iTUG only - Risk of Fall (Training set)", fontsize=20)
plt.bar([i for i in range(n_groups)], [100 *(i / j) for i, j in zip(xv, t)])
plt.xticks([i for i in range(n_groups)], ["Rank {0}".format(i+1) for i in range(n_groups)], fontsize=20)
plt.yticks(fontsize=16)
plt.ylabel("Risk of Falling [%]", fontsize=20)
plt.ylim(0, 100)

for i, v in enumerate(x):
    plt.text(i-0.2, 100*v/groups_l[i] + 5, str(np.around(100.0*v/groups_l[i])) + "%",
            color = 'blue', fontweight = 'bold', fontsize=18)

plt.savefig(BASE_DIR + "risk_of_fall_itug_only - training.png", dpi=300)

### Thresholds

In [ ]:
thresholds_dict_itug_list = {}

for i, var in enumerate(new_columns_itug[:-1]):
    if i != 0:
        ordered = dataset_i_tug_r.sort_values(var, ascending=ordering_itug[i]).copy()
        d_i = {}
        # Find thresholds
        th1 = dataset_i_tug_r.sort_values(new_columns_itug[i], ascending=ordering_itug[i]).copy().iloc[percentiles[0]][new_columns_itug[i]]
        th2 = dataset_i_tug_r.sort_values(new_columns_itug[i], ascending=ordering_itug[i]).copy().iloc[2*percentiles[0]][new_columns_itug[i]]

        d_i["Order"] = "Ascending" if ordering_itug[i] else "Descending"
        d_i["Th1"] = th1
        d_i["Th2"] = th2
        thresholds_dict_itug_list[var] = d_i

out_cols = ["Feature", "Rank1", "Rank2", "Rank3"]
out_df = pd.DataFrame(columns=out_cols)

rows = []
for key in thresholds_dict_itug_list.keys():
    if thresholds_dict_itug_list[key]["Order"] == "Descending":
        t1 = round(thresholds_dict_itug_list[key]["Th1"],2)
        t2 = round(thresholds_dict_itug_list[key]["Th2"],2)
        rows.append({"Feature": key, "Rank1": "x > {0}".format(t1), "Rank2": "{0} < x < {1}".format(t2, t1), "Rank3": "x < {0}".format(t2)})

for key in thresholds_dict_itug_list.keys():
    if thresholds_dict_itug_list[key]["Order"] == "Ascending":
        t1 = round(thresholds_dict_itug_list[key]["Th1"],2)
        t2 = round(thresholds_dict_itug_list[key]["Th2"],2)
        rows.append({"Feature": key, "Rank1": "x < {0}".format(t1), "Rank2": "{0} < x < {1}".format(t1, t2), "Rank3": "x > {0}".format(t2)})

out_df = pd.concat([out_df, pd.DataFrame(rows)], ignore_index=True)
#out_df.to_excel(BASE_DIR + "features_with_thresholds_itug.xlsx", index=False)

### Test

In [ ]:
import random

# Get relevant columns only
relevant_columns_itug = list(ordered_ranking_df_itug.columns)[:-1]
dataset_validation_itug_r = dataset_validation_itug[relevant_columns_itug]
dataset_validation_itug

# For each validation patient, perform classification and rank them
rankings_itug = {}
for i in range(len(dataset_validation_itug_r)):
  # Get row
  row_i = dataset_validation_itug_r.iloc[i]
  patient_scores = []
  # For each column, determing whether score is R1, R2 or R3
  for col in dataset_validation_itug_r.columns:
    if col in thresholds_dict_itug_list.keys():
      order = thresholds_dict_itug_list[col]['Order']
      th1 = thresholds_dict_itug_list[col]['Th1']
      th2 = thresholds_dict_itug_list[col]['Th2']
      # Check membership to rank based on order and threshold values
      score = None
      value = row_i[col]
      if order == "Descending":
        if value > th1:
          score = 0
        elif value > th2:
          score = 1
        else:
          score = 2
      elif order == "Ascending":
        if value < th1:
          score = 0
        elif value < th2:
          score = 1
        else:
          score = 2
      patient_scores.append(score)
  rankings_itug[row_i["Id paziente (H-Opera) "]] = max(set(patient_scores), key=patient_scores.count)
itug_rankings_list = []
for t in list(rankings_itug.items()):
    itug_rankings_list.append(list(t))
itug_ranking_df = pd.DataFrame(itug_rankings_list, columns=["Id paziente (H-Opera) ", "Ranking"])
dataset_validation_itug_r = pd.merge(dataset_validation_itug_r, itug_ranking_df, on='Id paziente (H-Opera) ')
#dataset_validation_itug_r

In [ ]:
#print("Validation dataset results:\n")

r1 = dataset_validation_itug_r[dataset_validation_itug_r['Ranking'] == 0]
r1_f = r1[r1['caduto almeno una volta'] == 1]
r1_nf = r1[r1['caduto almeno una volta'] == 0]

#print("{0} patients classified as Rank#1: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r1), len(r1_f), 100.0*round(float(len(r1_f)/len(r1)),2)))
#print("---")

r2 = dataset_validation_itug_r[dataset_validation_itug_r['Ranking'] == 1]
r2_f = r2[r2['caduto almeno una volta'] == 1]
r2_nf = r2[r2['caduto almeno una volta'] == 0]

#try:
#  print("{0} patients classified as Rank#2: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r2), len(r2_f), 100.0*round(float(len(r2_f)/len(r2)),2)))
#  print("---")
#except:
#  pass

r3 = dataset_validation_itug_r[dataset_validation_itug_r['Ranking'] == 2]
r3_f = r3[r3['caduto almeno una volta'] == 1]
r3_nf = r3[r3['caduto almeno una volta'] == 0]

#print("{0} patients classified as Rank#3: of these, {1} experienced at least one fall episode, with a risk of fall percentage {2}%".format(len(r3), len(r3_f), 100.0*round(float(len(r3_f)/len(r3)),2)))

group_l = [
    np.round(100.0*np.round(len(r1_f)/len(r1),2)),
    np.round(100.0*np.round(len(r2_f)/len(r2),2)),
    np.round(100.0*np.round(len(r3_f)/len(r3),2))]
n_groups = 3

plt.figure(figsize=(9,6))
plt.title("iTug only - Risk of fall (Test set)", fontsize=20)
plt.bar([i for i in range(n_groups)], group_l)
plt.xticks([i for i in range(n_groups)], ["Rank {0}".format(i+1) for i in range(n_groups)], fontsize=20)
plt.yticks(fontsize=16)
plt.ylabel("Risk of Falling [%]", fontsize=20)
plt.ylim(0, 100)

for i in range(n_groups):
    plt.text(i - 0.2, group_l[i] + 5, str(group_l[i]) + "%",
            color = 'blue', fontweight = 'bold', fontsize=18)

#plt.savefig(BASE_DIR + "risk_of_fall_itug_only - validation.png", dpi=300)